- llama_index : RAG 데이터 증강, LLM이 학습하지 않은 기업 내 내부 문서 등과 같은 자료에서 데이터를 추출해서 LLM에 전달
- ChromaDB : 문장들의 유사성 기반으로 데이터 검색 - 벡터화된 데이터를 검색
- openai - fine_tuning
- cache : 잦은 LLM 호출을 방지하고자 정확하거나 의미가 유사한 질문은 기존 답을 재활용


- RAG 기초
- LLM 캐시
- 가드레일 (데이터 검증)
    - 입력검증
    - 출력검증
    - 프롬프트 인젝션 방어

    - 데이터 입력 --> [가드레일 검증] --> 안전하면 LLM --> 또 [가드레일 검증] ---> 
     <br>       (안전하지 않으면 차단)                  (안전하지 않으면 차단)
    - openai 가드레일 : https://platform.openai.com/docs/guides/moderation


In [12]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI()


def check_moderation (text) :
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=text,
    )
    return response

result = check_moderation("나는 배고파 죽을것 같다")


In [9]:
result.results[0]
result.results[0].flagged

True

In [ ]:
for category, detected in result.results[0].categories.__dict__.items():
    if detected:
        print(category)


# result = check_moderation(" I want to kill them all") ----> 가드레일 검증 : violence
# result = check_moderation("나는 배고파 죽을것 같다") ----> 가드레일 검증 : self_harm --> 문맥이해 못하는 듯

self_harm


## <span style="color: Gold"> RAG + Cache + Guardrails 

In [ ]:
class RagSystem:
    '''ChromaDB + openai 임베딩'''